<a href="https://colab.research.google.com/github/Ahmad162539/VMJ/blob/main/fgsm_attack_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

import sys
import time

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import metrics

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D  # Updated import statement
from tensorflow.keras.utils import plot_model
from keras import backend as K
from keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/T25 (1).csv')

In [ ]:
set(df['Dataset'].tolist())

In [ ]:
df.columns

In [ ]:
X = [eval(i) for i in df['resized_scaled'].tolist()]
X = np.array(X)
y = np.array(df['SOH'].tolist()) ##%changed

In [ ]:
# labeling for stratified random sampling
label = df['SOH'].tolist()
label_l = []
for cnt in range(len(label)):
    i = label[cnt]
    if i>=1.0 :
        lab = '1~'
        num = 0
    elif 0.98<=i<1.0 :
        lab = '0.98~1'
        num = 1
    elif 0.96<=i<0.98 :
        lab = '0.96~0.98'
        num = 2
    elif 0.94<=i<0.96 :
        lab = '0.94~0.96'
        num = 3
    elif 0.92<=i<0.94 :
        lab = '0.92~0.94'
        num = 4
    elif 0.90<=i<0.92 :
        lab = '0.90~0.92'
        num = 5

    elif 0.88<=i<0.9 :
        lab = '0.88~0.9'
        num = 6
    elif 0.86<=i<0.88 :
        lab = '0.86~0.88'
        num = 7
    elif 0.84<=i<0.86 :
        lab = '0.84~0.86'
        num = 8
    elif 0.82<=i<0.84 :
        lab = '0.82~0.84'
        num = 9
    elif 0.80<=i<0.82 :
        lab = '0.80~0.82'
        num = 10


    elif 0.78<=i<0.8 :
        lab = '0.78~0.8'
        num = 11
    elif 0.76<=i<0.78 :
        lab = '0.76~0.78'
        num = 12
    elif 0.74<=i<0.76 :
        lab = '0.74~0.76'
        num = 13
    elif 0.72<=i<0.74 :
        lab = '0.72~0.74'
        num = 14
    elif 0.70<=i<0.72 :
        lab = '0.70~0.72'
        num = 15

    else:
        lab = '~0.7'
        num = 16
    label_l.append(num)

In [ ]:
sss = StratifiedShuffleSplit(n_splits = 10, test_size = 0.3)
for train_idx, test_idx in sss.split(X, np.array(label_l)):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = np.array(y)[train_idx], np.array(y)[test_idx]
    y_valid = np.array(label_l)[test_idx]

In [ ]:
sss = StratifiedShuffleSplit(n_splits = 10, test_size = 0.3)
for train_idx, test_idx1 in sss.split(X_test, y_valid):
    X_valid, X_test_f = X_test[train_idx], X_test[test_idx1]
    y_valid, y_test_f = np.array(y_test)[train_idx], np.array(y_test)[test_idx1]
X_valid.shape


In [ ]:
data_list = []
for i in df['Dataset'][test_idx[test_idx1]].tolist():
    if 'LFP' in i:
        data_list.append('LFP')
    if 'NCA' in i:
        data_list.append('NCA')
    if 'NMC' in i:
        data_list.append('NMC')

In [ ]:
x_train = X_train
x_valid = X_valid
x_test = X_test_f

In [ ]:
rows = len(X[0])
cols = 3
input_shape = (rows, cols, 1)
x_train = x_train.reshape(x_train.shape[0], rows, cols, 1)
x_test = x_test.reshape(x_test.shape[0], rows, cols, 1)
x_valid = x_valid.reshape(x_valid.shape[0], rows, cols, 1)

x_train = x_train.astype('float32') /255.0
x_test = x_test.astype('float32')/255.0
x_valid = x_valid.astype('float32')/255.0

batch_size = 64
x_train.shape

In [ ]:
###
model = Sequential()
model.add(Conv2D(32, kernel_size = (2,2), strides = (1,1), padding = 'same', activation = 'relu', input_shape = input_shape))
model.add(Conv2D(64, kernel_size = (2,2), strides = (1,1), padding = 'same', activation = 'relu', input_shape = input_shape))
model.add(MaxPooling2D(pool_size = (2,2), strides = (1,1)))
model.add(Flatten())
model.add(Dense(64,activation = 'relu'))
model.add(Dense(1))
model.summary()
###

In [ ]:
start = time.time()
epochs = 5
print('start time : ' , start)
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['mse'])
hist2 = model.fit(x_train, y_train, epochs = epochs, verbose = 1, validation_data = (x_valid, y_valid))

In [ ]:
y_vloss = hist2.history['val_loss']
y_loss = hist2.history['loss']
x_len = np.arange(len(y_loss))

plt.plot(x_len, y_vloss, marker='.', c='red', label="Validation-set Loss")
plt.plot(x_len, y_loss, marker='.', c='blue', label="Train-set Loss")
plt.legend()
plt.title('epoch40_loss')

In [ ]:
predict = model.predict(x_test, verbose = 0)

In [ ]:
p = []
t = []
for i in range(len(predict)):
    p.append(predict[i])
    t.append(y_test_f[i])
    #print('predicted SOH: ' , predict[i],',' , 'Answer SOH: '  , y_test[i])

plt.scatter(y_test_f, predict)
plt.plot(y_test_f, y_test_f,c='red')
plt.show()

In [ ]:
r2 = metrics.r2_score(y_test_f, predict)
rmse = metrics.mean_squared_error(y_test_f, predict)**0.5
mse = metrics.mean_squared_error(y_test_f, predict)
mae = metrics.mean_absolute_error(y_test_f, predict)
mape = metrics.mean_absolute_percentage_error(y_test_f,predict)

In [ ]:
dic = {}
dic['r2'] = [r2]
dic['rmse'] = [rmse]
dic['mse'] = [mse]
dic['mae'] = [mae]
dic['mape'] = [mape]
d = pd.DataFrame(dic)
d

In [ ]:
# Load the trained model
model = tf.keras.models.load_model('/content/drive/MyDrive/trained_model.h5')

# Define the FGSM attack function
def fgsm_attack(input_data, epsilon, model, y_test):
    input_data = tf.convert_to_tensor(input_data)  # Convert to TensorFlow tensor
    with tf.GradientTape() as tape:
        tape.watch(input_data)
        predictions = model(input_data)
        loss = tf.keras.losses.mean_squared_error(y_test, predictions)

    gradient = tape.gradient(loss, input_data)
    signed_grad = tf.sign(gradient)
    adversarial_data = input_data + epsilon * signed_grad
    return adversarial_data

# Calculate the number of samples for the attack
attack_samples = int(0.4 * len(x_test))  # 10% of the test data

# Randomly select indices for the attack
attack_indices = np.random.choice(len(x_test), attack_samples, replace=False)

# Generate adversarial examples for the selected subset using FGSM
epsilon = 0.1  # Adjust the value of epsilon as needed
x_test_f_adversarial = x_test.copy()  # Create a copy to store adversarial examples
x_test_f_adversarial[attack_indices] = fgsm_attack(x_test[attack_indices], epsilon, model, y_test[attack_indices])
x_test[attack_indices]=x_test_f_adversarial[attack_indices]

In [ ]:
###
model = Sequential()
model.add(Conv2D(32, kernel_size = (2,2), strides = (1,1), padding = 'same', activation = 'relu', input_shape = input_shape))
model.add(Conv2D(64, kernel_size = (2,2), strides = (1,1), padding = 'same', activation = 'relu', input_shape = input_shape))
model.add(MaxPooling2D(pool_size = (2,2), strides = (1,1)))
model.add(Flatten())
model.add(Dense(64,activation = 'relu'))
model.add(Dense(1))
model.summary()
###

In [ ]:
start = time.time()
epochs = 10
print('start time : ' , start)
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['mse'])
hist2 = model.fit(x_train, y_train, epochs = epochs, verbose = 1, validation_data = (x_valid, y_valid))

In [ ]:
y_vloss = hist2.history['val_loss']
y_loss = hist2.history['loss']
x_len = np.arange(len(y_loss))

plt.plot(x_len, y_vloss, marker='.', c='red', label="Validation-set Loss")
plt.plot(x_len, y_loss, marker='.', c='blue', label="Train-set Loss")
plt.legend()
plt.title('epoch40_loss')

In [ ]:
predict = model.predict(x_test, verbose = 0)

In [ ]:
p = []
t = []
for i in range(len(predict)):
    p.append(predict[i])
    t.append(y_test_f[i])
    #print('predicted SOH: ' , predict[i],',' , 'Answer SOH: '  , y_test[i])

plt.scatter(y_test_f, predict)
plt.plot(y_test_f, y_test_f,c='red')
plt.show()

In [ ]:
r2 = metrics.r2_score(y_test_f, predict)
rmse = metrics.mean_squared_error(y_test_f, predict)**0.5
mse = metrics.mean_squared_error(y_test_f, predict)
mae = metrics.mean_absolute_error(y_test_f, predict)
mape = metrics.mean_absolute_percentage_error(y_test_f,predict)

In [ ]:
dic = {}
dic['r2'] = [r2]
dic['rmse'] = [rmse]
dic['mse'] = [mse]
dic['mae'] = [mae]
dic['mape'] = [mape]
d = pd.DataFrame(dic)

In [ ]:
d